# Data Quality: Code Practice

In [ ]:
import pandas as pd
import numpy as np

from sklearn import datasets
from sklearn import ensemble

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataQualityPreset
from evidently.metrics import *
from evidently.test_suite import TestSuite
from evidently.test_preset import DataQualityTestPreset, DataStabilityTestPreset
from evidently.tests import *

In [ ]:
#Dataset for Data Quality and Integrity
adult_data = datasets.fetch_openml(name='adult', version=2, as_frame='auto')
adult = adult_data.frame

adult_ref = adult[~adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]
adult_cur = adult[adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]

adult_cur.iloc[:2000, 3:5] = np.nan

## Data Quality Test Suite

In [ ]:
data_quality = TestSuite(tests=[
    DataQualityTestPreset(),
])

data_quality.run(reference_data=adult_ref, current_data=adult_cur)
data_quality.show(mode='inline')

In [ ]:
#column-level tests
data_quality_column_tests = TestSuite(tests=[
    TestColumnValueMean(column_name='education-num'),
    TestColumnValueMedian(column_name='education-num'),
    TestNumberOfUniqueValues(column_name='education'),
    TestMostCommonValueShare(column_name='education'),
    TestValueRange(column_name='education-num'),
    TestShareOfOutRangeValues(column_name='education-num'),
    TestNumberOfOutListValues(column_name='education'),
    TestColumnQuantile(column_name='education-num', quantile=0.25),
    TestColumnShareOfMissingValues(column_name='education'),
    TestColumnRegExp(column_name='education',reg_exp='^[0..9]'),
    TestCategoryShare(column_name='education', category='Some-college', lt=0.5),
])

data_quality_column_tests.run(reference_data=adult_ref, current_data=adult_cur)
data_quality_column_tests.show(mode='inline')

## Data Quality Report

In [ ]:
data_quality_report = Report(metrics=[
    DataQualityPreset(),
])

data_quality_report.run(reference_data=adult_ref, current_data=adult_cur)
data_quality_report.show(mode='inline')

In [ ]:
#dataset-level metrics
data_quality_dataset_report = Report(metrics=[
    DatasetSummaryMetric(),
    DatasetMissingValuesMetric(),
    DatasetCorrelationsMetric(),
    
])

data_quality_dataset_report.run(reference_data=adult_ref, current_data=adult_cur)
data_quality_dataset_report.show(mode='inline')

In [ ]:
data_quality_column_report = Report(metrics=[
    ColumnDistributionMetric(column_name="education"), 
    ColumnQuantileMetric(column_name="education-num", quantile=0.75), 
    ColumnCorrelationsMetric(column_name="education"),
    ColumnValueListMetric(column_name="relationship", values=["Husband", "Unmarried"]), 
    ColumnValueRangeMetric(column_name="age", left=10, right=20),
    ColumnCategoryMetric(column_name='education', category='Some-college'),
    ColumnRegExpMetric(column_name="relationship", reg_exp=r".*child.*"),
    ColumnSummaryMetric(column_name="age"),
    ColumnMissingValuesMetric(column_name="education"),    
])

data_quality_column_report.run(reference_data=adult_ref, current_data=adult_cur)
data_quality_column_report.show(mode='inline')